<a href="https://colab.research.google.com/github/Nareshedagotti/RAG/blob/main/StandardRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain groq faiss-cpu sentence-transformers PyPDF2 python-docx langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from PyPDF2 import PdfReader
from docx import Document

def extract_text(file_paths):
    documents = []
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = "".join(page.extract_text() or "" for page in pdf_reader.pages)
                documents.append({"content": text, "source": file_path})
        elif file_path.endswith('.docx'):
            doc = Document(file_path)
            text = "\n".join(para.text for para in doc.paragraphs)
            documents.append({"content": text, "source": file_path})
    return documents

file_path = ["/content/Indexing.pdf" ]
text = extract_text(file_path)
print(text)

[{'content': "VECTOR DATABASE \nINDEXING\nSTATFUSIONAISwipe\nSwipe\nWhat is Indexing in VectorDBs?\nIndexing is a fundamental technique used to organize vector data for efficient \nsimilarity search operations. In the context of vector databases, indexing creates \nspecialized data structures that make nearest neighbor search operations faster \nand more efficient, especially when dealing with high-dimensional embeddings \nfrom text, images, audio, or other data types.\nWhy Indexing Matters\n•Raw vector comparison (brute force) becomes prohibitively slow as datasets \ngrow\n•Dimensionality curse makes searching in high-dimensional spaces challenging\n•Properly indexed data can deliver results orders of magnitude faster\n•Enables practical applications of vector search in production environments\nTwo Major Categories of Indexing\n1.Exact Indexing\n•Guarantees 100% accurate results\n•Generally slower but provides perfect recall\n•Example: Flat (brute-force) index\n2.   Approximate Indexi

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunks = []
    for doc in documents:
        split_docs = text_splitter.create_documents(
            [doc["content"]],
            metadatas=[{"source": doc["source"]}]
        )
        chunks.extend(split_docs)
    return chunks

chunks = chunk_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"\n🔹 **Chunk {i}:**\n{chunk.page_content}\n{'-'*50}")


🔹 **Chunk 1:**
VECTOR DATABASE 
INDEXING
STATFUSIONAISwipe
Swipe
What is Indexing in VectorDBs?
Indexing is a fundamental technique used to organize vector data for efficient 
similarity search operations. In the context of vector databases, indexing creates 
specialized data structures that make nearest neighbor search operations faster 
and more efficient, especially when dealing with high-dimensional embeddings 
from text, images, audio, or other data types.
Why Indexing Matters
•Raw vector comparison (brute force) becomes prohibitively slow as datasets 
grow
•Dimensionality curse makes searching in high-dimensional spaces challenging
•Properly indexed data can deliver results orders of magnitude faster
•Enables practical applications of vector search in production environments
Two Major Categories of Indexing
1.Exact Indexing
•Guarantees 100% accurate results
•Generally slower but provides perfect recall
•Example: Flat (brute-force) index
------------------------------------------

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

def generate_embeddings(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    return embedding_model, chunks

embedding_model, chunks = generate_embeddings(chunks)

<ipython-input-4-61f3aaff5793>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from langchain.vectorstores import FAISS

def store_in_db(chunks, embedding_model):
    vector_db = FAISS.from_documents(chunks, embedding_model)
    return vector_db

vector_db = store_in_db(chunks, embedding_model)

In [6]:
def query_to_embeddings(query, embedding_model):
    query_embedding = embedding_model.embed_query(query)
    return query_embedding

query = "give me different types of vector indexing and uses?"
query_embedding = query_to_embeddings(query, embedding_model)


In [7]:
def retrieve_top_matches(vector_db, query_embedding):
    top_matches = vector_db.similarity_search_by_vector(query_embedding, k=3)
    return top_matches

top_matches = retrieve_top_matches(vector_db, query_embedding)
print(top_matches)

[Document(id='f5d4b59a-b677-4caf-a35d-916972f74aec', metadata={'source': '/content/Indexing.pdf'}, page_content='VECTOR DATABASE \nINDEXING\nSTATFUSIONAISwipe\nSwipe\nWhat is Indexing in VectorDBs?\nIndexing is a fundamental technique used to organize vector data for efficient \nsimilarity search operations. In the context of vector databases, indexing creates \nspecialized data structures that make nearest neighbor search operations faster \nand more efficient, especially when dealing with high-dimensional embeddings \nfrom text, images, audio, or other data types.\nWhy Indexing Matters\n•Raw vector comparison (brute force) becomes prohibitively slow as datasets \ngrow\n•Dimensionality curse makes searching in high-dimensional spaces challenging\n•Properly indexed data can deliver results orders of magnitude faster\n•Enables practical applications of vector search in production environments\nTwo Major Categories of Indexing\n1.Exact Indexing\n•Guarantees 100% accurate results\n•Genera

In [8]:
from groq import Groq

def pass_to_llm(query, top_matches, api_key):
    context = "\n\n".join(match.page_content for match in top_matches)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gemma2-9b-it"
    )
    return chat_completion.choices[0].message.content, top_matches


api_key="your_api_key"
llm_response, top_matches = pass_to_llm(query, top_matches, api_key)
print(llm_response)

Here's a breakdown of common vector indexing types and their uses, based on your provided context:

**1. Exact Indexing**

* **What it is:**  Guarantees finding the *exact* nearest neighbors to a given query vector. Think of it like looking up a specific word in a dictionary – you get a precise match.
* **Trade-offs:**
    * **Slower:**  More computationally expensive, especially as datasets grow large.
    * **High accuracy:** Provides perfect recall (you find all the truly closest vectors).
* **Example:**
    * **Flat Index (Brute-Force):**  
        * **How it works:** Compares the query vector to every vector in the database. 
        * **Use case:**  Small datasets where speed isn't critical and absolute accuracy is paramount.

**2. Approximate Indexing (ANN - Approximate Nearest Neighbors)**

* **What it is:** Finds *approximate* nearest neighbors. It's like finding words that are *similar* to your query word – not exact matches. It trades off some accuracy for significantly fast